In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/val/val_week1_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/test/test_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/clean_data_week2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_2.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_4.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_1.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week2/train/5-folds/data_part_5.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/val/val_week1_2_3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/test/test_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw_data/clean_week3/train/clean_data_week3.csv
/kaggle/input/new-raw-mooccubex/new_raw/raw

In [2]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import StratifiedKFold
import time
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support

<ipython-input-2-cd41008c27ef>:7: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
five_fold_files = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_5.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/5-folds/data_part_5.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/5-folds/data_part_5.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_1.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_2.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_3.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_4.csv",
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/5-folds/data_part_5.csv"
    ]
}

file_validataion = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/val/val_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/val/val_week1_2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/val/val_week1_2_3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/val/val_week1_2_3_4.csv"
    ]
}

file_test = {
    "week1": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/test/test_week1.csv"
    ],
    "week2": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/test/test_week2.csv"
    ],
    "week3": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/test/test_week3.csv"
    ],
    "week4": [
        "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/test/test_week4.csv"
    ]
}

## Tìm siêu tham số tốt nhất cho từng tuần

In [4]:
# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# Tạo hàm train cho từng tuần
def train_week_model(week_number, file_paths_train, file_validataion):
    # Đọc dữ liệu
    train_data = pd.read_csv(file_paths_train)
    val_data = pd.read_csv(file_validataion)
    
    # Tách đặc trưng và nhãn
    X_train = train_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_train = train_data["classification_encoded"]

    X_val = val_data.drop(columns=["classification_encoded", "user_id", "course_id", "school", "enroll_time", "classification"])
    y_val = val_data["classification_encoded"]
    
    # Áp dụng Over-sampling cho dữ liệu huấn luyện bằng SMOTE
    oversampler = SMOTE(sampling_strategy='auto', random_state=42)
    X_train_res, y_train_res = oversampler.fit_resample(X_train, y_train)
    
    # Reshape dữ liệu cho mô hình BiLSTM
    X_train_res = X_train_res.values.reshape(X_train_res.shape[0], X_train_res.shape[1], 1)
    X_val = X_val.values.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    # One-hot encode nhãn
    y_train_res = tf.keras.utils.to_categorical(y_train_res, num_classes=5)
    y_val = tf.keras.utils.to_categorical(y_val, num_classes=5)
    
    # Xây dựng mô hình BiLSTM
    def build_model(hp):
        model = tf.keras.Sequential()
        
        # Bidirectional LSTM layer 1
        model.add(layers.Bidirectional(layers.LSTM(
            units=hp.Int('units_1', min_value=32, max_value=256, step=32),
            return_sequences=True
        ), input_shape=(X_train_res.shape[1], 1)))
        model.add(layers.Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
        
        # Bidirectional LSTM layer 2
        model.add(layers.Bidirectional(layers.LSTM(
            units=hp.Int('units_2', min_value=32, max_value=256, step=32),
            return_sequences=False
        )))
        model.add(layers.Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
        
        # Lớp đầu ra
        model.add(layers.Dense(5, activation='softmax'))
        
        # Compile với Focal Loss
        model.compile(optimizer=tf.keras.optimizers.Adam(
                          learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss=focal_loss(gamma=2., alpha=0.25),
                      metrics=['accuracy'])
        return model
    
    # Khởi tạo RandomSearch tuner
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=5,
        executions_per_trial=1,
        directory='my_dir',
        project_name=f'bilstm_tuning_week{week_number}'
    )
    
    # Tìm kiếm siêu tham số tốt nhất
    tuner.search(X_train_res, y_train_res,
                 epochs=20,
                 validation_data=(X_val, y_val),
                 batch_size=32)
    
    # Trả về kết quả tối ưu cho tuần
    best_params = tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_params

In [5]:
# Định nghĩa đường dẫn đến dữ liệu cho từng tuần
file_paths_train = {
    "week1": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/clean_data_week1.csv",
    "week2": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/train/clean_data_week2.csv",
    "week3": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/train/clean_data_week3.csv",
    "week4": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/train/clean_data_week4.csv"
}

file_validataion = {
    "week1": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/val/val_week1.csv",
    "week2": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week2/val/val_week1_2.csv",
    "week3": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week3/val/val_week1_2_3.csv",
    "week4": "/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week4/val/val_week1_2_3_4.csv"
}

In [6]:
# Tìm tham số tốt nhất cho từng tuần
best_params_week1 = train_week_model(1, file_paths_train["week1"], file_validataion["week1"])
best_params_week2 = train_week_model(2, file_paths_train["week2"], file_validataion["week2"])
best_params_week3 = train_week_model(3, file_paths_train["week3"], file_validataion["week3"])
best_params_week4 = train_week_model(4, file_paths_train["week4"], file_validataion["week4"])

# In thông tin chi tiết các tham số tối ưu
print("Best Parameters for Week 1:")
for param_name in best_params_week1.values.keys():
    print(f"{param_name}: {best_params_week1.get(param_name)}")

print("\nBest Parameters for Week 2:")
for param_name in best_params_week2.values.keys():
    print(f"{param_name}: {best_params_week2.get(param_name)}")

print("\nBest Parameters for Week 3:")
for param_name in best_params_week3.values.keys():
    print(f"{param_name}: {best_params_week3.get(param_name)}")

print("\nBest Parameters for Week 4:")
for param_name in best_params_week4.values.keys():
    print(f"{param_name}: {best_params_week4.get(param_name)}")


Trial 5 Complete [00h 04m 53s]
val_accuracy: 0.9908480644226074

Best val_accuracy So Far: 0.9981696009635925
Total elapsed time: 00h 21m 44s
Best Parameters for Week 1:
units_1: 224
dropout_1: 0.4
units_2: 128
dropout_2: 0.30000000000000004
learning_rate: 0.0021299066766227647

Best Parameters for Week 2:
units_1: 160
dropout_1: 0.5
units_2: 160
dropout_2: 0.5
learning_rate: 0.0011567819399700755

Best Parameters for Week 3:
units_1: 256
dropout_1: 0.1
units_2: 96
dropout_2: 0.5
learning_rate: 0.0008151332617212836

Best Parameters for Week 4:
units_1: 160
dropout_1: 0.1
units_2: 160
dropout_2: 0.2
learning_rate: 0.0007558360818294099


## Danh sách tham số tốt nhất của từng tuần

In [7]:
# Danh sách tham số tốt nhất
best_params = {
    "week1": best_params_week1,
    "week2": best_params_week2,
    "week3": best_params_week3,
    "week4": best_params_week4
}

In [8]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_Bilstm_model(params):
    model = tf.keras.Sequential()
        
    # Bidirectional LSTM layer 1
    model.add(layers.Bidirectional(layers.LSTM(
        units=params.get('units_1'),
        return_sequences=True
    ), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))
    
    # Bidirectional LSTM layer 2
    model.add(layers.Bidirectional(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=False
    )))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))
    
    # Lớp đầu ra
    model.add(layers.Dense(5, activation='softmax'))
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy'])
    return model


In [9]:
# Biến lưu kết quả tổng quát
overall_results_5folds = []

# Lặp qua từng tuần
for week, file_paths in five_fold_files.items():
    print(f"\nProcessing {week} with best parameters...")
    params = best_params[week].values
    print(f"best parameters for {week}: {params}")
    
    # Biến lưu kết quả cho từng tuần
    week_results = {
        "week": week,
        "accuracy_per_fold": [],
        "precision_per_label": [],
        "recall_per_label": [],
        "f1_score_per_label": [],
        "confusion_matrices": [],
        "train_times": [],
        "test_times": []
    }

    # Lặp qua từng fold
    for i in range(len(file_paths)):
        print(f"Fold {i+1}: Using file {file_paths[i]} as test set")
        
        # Tải dữ liệu
        test_data = pd.read_csv(file_paths[i])
        train_data = pd.concat([pd.read_csv(file_paths[j]) for j in range(len(file_paths)) if j != i])
        
        # Tách X và y
        X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                           "course_id", "school", "enroll_time", "classification"])
        y_train = to_categorical(train_data['classification_encoded'], num_classes=5)
        
        X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                         "course_id", "school", "enroll_time", "classification"])
        y_test = to_categorical(test_data['classification_encoded'], num_classes=5)

        # Reshape dữ liệu cho LSTM
        X_train = X_train.to_numpy().reshape((X_train.shape[0], 1, X_train.shape[1]))
        X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

        # Xây dựng mô hình với tham số tốt nhất
        model = build_Bilstm_model(params)
        
        # Bắt đầu tính thời gian huấn luyện
        start_train = time.time()
        model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)
        end_train = time.time()
        
        # Bắt đầu tính thời gian kiểm thử
        start_test = time.time()
        y_pred = model.predict(X_test)
        end_test = time.time()
        
        # Tính thời gian và lưu lại
        train_time = end_train - start_train
        test_time = end_test - start_test
        week_results["train_times"].append(train_time)
        week_results["test_times"].append(test_time)

        # Đánh giá mô hình trên tập kiểm thử của fold hiện tại
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        week_results["accuracy_per_fold"].append(accuracy)
        
        # Dự đoán
        y_pred_classes = y_pred.argmax(axis=1)
        y_test_classes = y_test.argmax(axis=1)
        
        # Tính các chỉ số cho mỗi fold
        precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
        conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
        
        week_results["precision_per_label"].append(precision)
        week_results["recall_per_label"].append(recall)
        week_results["f1_score_per_label"].append(f1)
        week_results["confusion_matrices"].append(conf_matrix)

    # Tính trung bình cho từng nhãn
    average_precision_per_label = np.mean(week_results["precision_per_label"], axis=0)
    average_recall_per_label = np.mean(week_results["recall_per_label"], axis=0)
    average_f1_per_label = np.mean(week_results["f1_score_per_label"], axis=0)
    average_confusion_matrix = np.mean(week_results["confusion_matrices"], axis=0)
    average_train_time = sum(week_results["train_times"]) / len(week_results["train_times"])
    average_test_time = sum(week_results["test_times"]) / len(week_results["test_times"])
    
    # Tạo DataFrame cho precision, recall, f1-score
    labels = np.unique(y_test_classes)  # Lấy nhãn từ y_test_classes
    metrics_df = pd.DataFrame({
        "Label": labels,
        "Average Precision": average_precision_per_label,
        "Average Recall": average_recall_per_label,
        "Average F1-Score": average_f1_per_label
    })
    
    # Tạo DataFrame cho confusion matrix
    confusion_df = pd.DataFrame(average_confusion_matrix, index=labels, columns=labels)
    
    # In kết quả
    print("\n=== Average Precision, Recall, F1-Score per Label ===")
    print(metrics_df)
    print("\n=== Average Confusion Matrix ===")
    print(confusion_df)

    # Lưu kết quả tuần vào kết quả tổng quát
    week_results["average_train_times"] = average_train_time
    week_results["average_test_times"] = average_test_time
    week_results["average_metrics_df"] = metrics_df
    week_results["average_confusion_matrix"] = confusion_df
    overall_results_5folds.append(week_results)


Processing week1 with best parameters...
best parameters for week1: {'units_1': 224, 'dropout_1': 0.4, 'units_2': 128, 'dropout_2': 0.30000000000000004, 'learning_rate': 0.0021299066766227647}
Fold 1: Using file /kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_1.csv as test set
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6276 - loss: 0.1362 - val_accuracy: 0.6942 - val_loss: 0.1130
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6601 - loss: 0.1215 - val_accuracy: 0.6862 - val_loss: 0.1148
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6715 - loss: 0.1190 - val_accuracy: 0.6862 - val_loss: 0.1131
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6656 - loss: 0.1227 - val_accuracy: 0.6775 - val_loss: 0.1132
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6688 - loss: 0.1189 - val_accuracy: 0.6996 - val_loss: 0.1125
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6839 - loss: 0.1147 - val_accuracy: 0.7167 - val_loss: 0.1072
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6819 - loss: 0.1145 - val_accuracy: 0.7042 - val_loss: 0.1077
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6850 - loss: 0.1167 - val_accuracy: 0.7186 - val_

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Fold 2: Using file /kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week1/train/5-folds/data_part_2.csv as test set
Epoch 1/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6291 - loss: 0.1339 - val_accuracy: 0.6775 - val_loss: 0.1190
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6625 - loss: 0.1232 - val_accuracy: 0.6839 - val_loss: 0.1152
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6684 - loss: 0.1206 - val_accuracy: 0.6977 - val_loss: 0.1172
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6627 - loss: 0.1215 - val_accuracy: 0.6439 - val_loss: 0.1134
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6727 - loss: 0.1161 - val_accuracy: 0.7026 - val_loss: 0.1132
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6845 - loss: 0.1159 - val_accuracy: 0.7320 - val_loss: 0.1048
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6839 - loss: 0.1138 - val_accuracy: 0.6653

...

328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6637 - loss: 0.1243 - val_accuracy: 0.7490 - val_loss: 0.0996
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7743 - loss: 0.0895 - val_accuracy: 0.8009 - val_loss: 0.0749
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8530 - loss: 0.0627 - val_accuracy: 0.8879 - val_loss: 0.0491
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8780 - loss: 0.0542 - val_accuracy: 0.8780 - val_loss: 0.0530
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8923 - loss: 0.0457 - val_accuracy: 0.8871 - val_loss: 0.0448
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9032 - loss: 0.0394 - val_accuracy: 0.9222 - val_loss: 0.0292
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9170 - loss: 0.0333 - val_accuracy: 0.9382 - val_loss: 0.0230
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9238 - loss: 0.0274 - val_accuracy: 0.9409 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.6509 - loss: 0.1304 - val_accuracy: 0.7502 - val_loss: 0.0947
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7782 - loss: 0.0873 - val_accuracy: 0.8539 - val_loss: 0.0625
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8603 - loss: 0.0594 - val_accuracy: 0.8787 - val_loss: 0.0546
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8752 - loss: 0.0523 - val_accuracy: 0.9005 - val_loss: 0.0417
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8836 - loss: 0.0478 - val_accuracy: 0.8944 - val_loss: 0.0548
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8999 - loss: 0.0422 - val_accuracy: 0.9310 - val_loss: 0.0245
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9120 - loss: 0.0337 - val_accuracy: 0.9279 - val_loss: 0.0269
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9260 - loss: 0.0278 - val_accuracy: 0.9489 - val_

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


328/328 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.6621 - loss: 0.1272 - val_accuracy: 0.7662 - val_loss: 0.0979
Epoch 2/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7939 - loss: 0.0855 - val_accuracy: 0.8146 - val_loss: 0.0759
Epoch 3/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8456 - loss: 0.0629 - val_accuracy: 0.8852 - val_loss: 0.0461
Epoch 4/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8787 - loss: 0.0491 - val_accuracy: 0.9027 - val_loss: 0.0371
Epoch 5/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.8922 - loss: 0.0462 - val_accuracy: 0.9252 - val_loss: 0.0331
Epoch 6/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9112 - loss: 0.0371 - val_accuracy: 0.9283 - val_loss: 0.0293
Epoch 7/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9114 - loss: 0.0345 - val_accuracy: 0.9424 - val_loss: 0.0227
Epoch 8/50
328/328 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9222 - loss: 0.0291 - val_accuracy: 0.9420 - val

## Kết quả cross validation trên 5-folds

In [10]:
# Duyệt qua các tuần trong overall_results
for week_result in overall_results_5folds:
    week = week_result["week"]
    average_train_time = np.mean(week_result["train_times"])
    average_test_time = np.mean(week_result["test_times"])
    average_metrics_df = week_result["average_metrics_df"]
    average_confusion_matrix = week_result["average_confusion_matrix"]
    
    # In kết quả
    print(f"\n=== Results for {week} ===")
    print(f"Average Train Time: {average_train_time:.4f} seconds")
    print(f"Average Test Time: {average_test_time:.4f} seconds")
    print("\nAverage Precision, Recall, F1-Score per Label:")
    print(average_metrics_df)
    print("\nAverage Confusion Matrix:")
    print(average_confusion_matrix)



=== Results for week1 ===
Average Train Time: 128.2295 seconds
Average Test Time: 0.7855 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.703269        0.627667          0.661383
1      1           0.000000        0.000000          0.000000
2      2           0.670665        0.482882          0.547967
3      3           0.583626        0.476647          0.518157
4      4           0.835606        0.949229          0.888583

Average Confusion Matrix:
       0    1     2     3       4
0  376.6  0.0  17.8  30.4   175.2
1   36.0  0.0   2.4   9.6    39.6
2   39.0  0.0  79.4   8.8    37.2
3   36.2  0.0   3.0  79.6    48.2
4   49.6  0.0  20.4  11.4  1522.0

=== Results for week2 ===
Average Train Time: 128.2207 seconds
Average Test Time: 0.7667 seconds

Average Precision, Recall, F1-Score per Label:
   Label  Average Precision  Average Recall  Average F1-Score
0      0           0.857191        0.913667

## Kiểm tra trên tập test

In [11]:
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import backend as K

# Định nghĩa Focal Loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=-1)
    
    return focal_loss_fixed

# Xây dựng mô hình
def build_Bilstm_model(params):
    model = tf.keras.Sequential()
        
    # Bidirectional LSTM layer 1
    model.add(layers.Bidirectional(layers.LSTM(
        units=params.get('units_1'),
        return_sequences=True
    ), input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.Dropout(rate=params.get('dropout_1', 0.2)))
    
    # Bidirectional LSTM layer 2
    model.add(layers.Bidirectional(layers.LSTM(
        units=params.get('units_2', 32),
        return_sequences=False
    )))
    model.add(layers.Dropout(rate=params.get('dropout_2', 0.2)))
    
    # Lớp đầu ra
    model.add(layers.Dense(5, activation='softmax'))
    
    # Compile với Focal Loss
    model.compile(optimizer=tf.keras.optimizers.Adam(
                      learning_rate=params['learning_rate']),
                  loss=focal_loss(gamma=params.get('gamma', 2.), alpha=params.get('alpha', 0.25)),
        metrics=['accuracy'])
    return model


In [12]:
# Mảng lưu dữ liệu của các tuần
results = []

def process_week(week_num, best_params, results):
    print(f"\n=== Processing Week {week_num} ===")
    params = best_params[week].values
    # Đường dẫn tới dữ liệu tuần tương ứng
    train_path = f"/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week{week_num}/train/clean_data_week{week_num}.csv"
    test_path = f"/kaggle/input/new-raw-mooccubex/new_raw/clean_data/clean_week{week_num}/test/test_week{week_num}.csv"
    
    # Load dữ liệu
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)
    
    # Tách X và y
    X_train = train_data.drop(columns=["classification_encoded", "user_id",
                                       "course_id", "school", "enroll_time", "classification"])
    y_train = train_data['classification_encoded']
    
    X_test = test_data.drop(columns=["classification_encoded", "user_id",
                                     "course_id", "school", "enroll_time", "classification"])
    y_test = test_data['classification_encoded']

    # Áp dụng SMOTE cho tập huấn luyện
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Chuyển đổi nhãn sang dạng one-hot
    y_train_resampled = to_categorical(y_train_resampled, num_classes=5)
    y_test = to_categorical(y_test, num_classes=5)

    # Reshape dữ liệu cho LSTM
    X_train_resampled = X_train_resampled.to_numpy().reshape((X_train_resampled.shape[0], 1, X_train_resampled.shape[1]))
    X_test = X_test.to_numpy().reshape((X_test.shape[0], 1, X_test.shape[1]))

    # Xây dựng mô hình với tham số tốt nhất
    model = build_Bilstm_model(params)
    
    # Huấn luyện mô hình
    start_train = time.time()
    model.fit(X_train_resampled, y_train_resampled, epochs=50, validation_split=0.1, batch_size=32)
    end_train = time.time()
    
    # Kiểm thử mô hình
    start_test = time.time()
    y_pred = model.predict(X_test)
    end_test = time.time()
    
    # Tính thời gian huấn luyện và kiểm thử
    train_time = end_train - start_train
    test_time = end_test - start_test
    
    # Đánh giá mô hình
    y_pred_classes = y_pred.argmax(axis=1)
    y_test_classes = y_test.argmax(axis=1)
    
    precision, recall, f1, _ = precision_recall_fscore_support(y_test_classes, y_pred_classes, average=None)
    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
    
    # Lưu kết quả vào mảng
    results.append({
        "week": week_num,
        "train_time": train_time,
        "test_time": test_time,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "confusion_matrix": conf_matrix
    })
    
    print("\n=== Precision, Recall, F1-Score per Label ===")
    print(pd.DataFrame({
        "Label": np.unique(y_test_classes),
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    }))
    print("\n=== Confusion Matrix ===")
    print(pd.DataFrame(conf_matrix, index=np.unique(y_test_classes), columns=np.unique(y_test_classes)))
    
    print(f"\nTrain Time: {train_time:.2f} seconds")
    print(f"Test Time: {test_time:.2f} seconds")

In [13]:
process_week(1, best_params, results)


=== Processing Week 1 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.5325 - loss: 0.1812 - val_accuracy: 0.5313 - val_loss: 0.2861
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6209 - loss: 0.1478 - val_accuracy: 0.6812 - val_loss: 0.1966
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6526 - loss: 0.1325 - val_accuracy: 0.6914 - val_loss: 0.1517
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6758 - loss: 0.1209 - val_accuracy: 0.5954 - val_loss: 0.1955
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6927 - loss: 0.1131 - val_accuracy: 0.6191 - val_loss: 0.2026
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.7045 - loss: 0.1077 - val_accuracy: 0.6687 - val_loss: 0.1494
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.7319 - loss: 0.0968 - val_accuracy: 0.6755 - val_loss: 0.1113
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.7432 - loss: 0.0879 - val_accura

In [14]:
process_week(2, best_params, results)


=== Processing Week 2 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.5416 - loss: 0.1774 - val_accuracy: 0.6081 - val_loss: 0.2135
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6637 - loss: 0.1231 - val_accuracy: 0.7092 - val_loss: 0.1010
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.7475 - loss: 0.0844 - val_accuracy: 0.9945 - val_loss: 0.0067
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8065 - loss: 0.0586 - val_accuracy: 0.9940 - val_loss: 0.0045
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8351 - loss: 0.0467 - val_accuracy: 0.9878 - val_loss: 0.0070
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8506 - loss: 0.0389 - val_accuracy: 1.0000 - val_loss: 3.3904e-04
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8604 - loss: 0.0362 - val_accuracy: 0.9985 - val_loss: 0.0010
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8702 - loss: 0.0321 - val_ac

In [15]:
process_week(3, best_params, results)


=== Processing Week 3 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.5414 - loss: 0.1744 - val_accuracy: 0.5685 - val_loss: 0.2096
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.7450 - loss: 0.0894 - val_accuracy: 0.9429 - val_loss: 0.0209
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8263 - loss: 0.0515 - val_accuracy: 0.7004 - val_loss: 0.1974
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8649 - loss: 0.0346 - val_accuracy: 0.9965 - val_loss: 0.0036
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8826 - loss: 0.0290 - val_accuracy: 0.9803 - val_loss: 0.0065
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8944 - loss: 0.0226 - val_accuracy: 0.9998 - val_loss: 5.2749e-04
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9007 - loss: 0.0212 - val_accuracy: 1.0000 - val_loss: 8.1241e-04
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8994 - loss: 0.0220 - va

In [16]:
process_week(4, best_params, results)


=== Processing Week 4 ===
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1128/1128 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.5477 - loss: 0.1727 - val_accuracy: 0.6021 - val_loss: 0.2015
Epoch 2/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.7696 - loss: 0.0779 - val_accuracy: 0.9204 - val_loss: 0.0234
Epoch 3/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.8562 - loss: 0.0386 - val_accuracy: 0.9651 - val_loss: 0.0100
Epoch 4/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.8841 - loss: 0.0270 - val_accuracy: 0.9943 - val_loss: 0.0018
Epoch 5/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9013 - loss: 0.0203 - val_accuracy: 0.9965 - val_loss: 0.0018
Epoch 6/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9084 - loss: 0.0171 - val_accuracy: 0.9920 - val_loss: 0.0026
Epoch 7/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.9123 - loss: 0.0172 - val_accuracy: 1.0000 - val_loss: 6.2543e-05
Epoch 8/50
1128/1128 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9172 - loss: 0.0160 - val_ac

In [17]:
# Hiển thị dữ liệu của các tuần
print("\n=== Summary Results for All Weeks ===")
for result in results:
    print(f"Week {result['week']}:")
    print(f"  Train Time: {result['train_time']:.2f} seconds")
    print(f"  Test Time: {result['test_time']:.2f} seconds")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1-Score: {result['f1_score']}")
    print(f"  Confusion Matrix:\n{result['confusion_matrix']}")


=== Summary Results for All Weeks ===
Week 1:
  Train Time: 413.80 seconds
  Test Time: 0.70 seconds
  Precision: [0.99421965 0.43103448 0.70149254 0.84883721 0.97390397]
  Recall: [0.91733333 0.92592593 0.91262136 0.6952381  0.93020937]
  F1-Score: [0.95423024 0.58823529 0.79324895 0.76439791 0.95155533]
  Confusion Matrix:
[[344   3   3   0  25]
 [  0  50   4   0   0]
 [  0   0  94   9   0]
 [  0   2  30  73   0]
 [  2  61   3   4 933]]
Week 2:
  Train Time: 423.75 seconds
  Test Time: 0.68 seconds
  Precision: [0.99457995 0.86206897 0.94392523 1.         1.        ]
  Recall: [0.97866667 0.92592593 0.98058252 1.         0.99800598]
  F1-Score: [0.98655914 0.89285714 0.96190476 1.         0.999002  ]
  Confusion Matrix:
[[ 367    5    3    0    0]
 [   1   50    3    0    0]
 [   0    2  101    0    0]
 [   0    0    0  105    0]
 [   1    1    0    0 1001]]
Week 3:
  Train Time: 422.31 seconds
  Test Time: 0.69 seconds
  Precision: [1.         0.87096774 0.98076923 1.         1.   